![dvd_image](dvd_image.jpg)

A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

In [25]:
!python3.11 -m pip install --upgrade pip
!pip install -U scikit-learn
!pip install scikit-learn-extra
!pip install --upgrade scikit-learn

In [26]:
# Import any additional modules and start coding below
import pandas as pd
import numpy as np

# Import scikit learn
import sklearn

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

from sklearn.linear_model import LogisticRegression

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor

from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostClassifier

In [27]:
# Read in the csv file
df = pd.read_csv('rental_info.csv')
SEED = 9
NESTIM = 300

df.head()

,rental_date,return_date,amount,release_year,rental_rate,length,replacement_cost,special_features,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2
0,2005-05-25 02:54:33+00:00,2005-05-28 23:40:33+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
1,2005-06-15 23:19:16+00:00,2005-06-18 19:24:16+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
2,2005-07-10 04:27:45+00:00,2005-07-17 10:11:45+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
3,2005-07-31 12:06:41+00:00,2005-08-02 14:30:41+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
4,2005-08-19 12:30:04+00:00,2005-08-23 13:35:04+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401


In [28]:
# Create a culumn to show number of days each DVD has been rented
df['rental_length_days'] = (pd.to_datetime(df['return_date']) - pd.to_datetime(df['rental_date'])).dt.days

In [29]:
# Create two dummy variables to indicate 'Deleted Scens' and 'Behind the Scenes'
df['deleted_scenes'] = np.where(df['special_features'].str.contains('Deleted Scenes'), 1, 0)
df['behind_the_scenes'] = np.where(df['special_features'].str.contains('Behind the Scenes'), 1, 0)
print(df['deleted_scenes'].value_counts())

deleted_scenes
0    7973
1    7888
Name: count, dtype: int64


In [30]:
# Make a data frame for features and a list for 
features_to_keep = ['amount', 'amount_2', 'rental_rate', 'rental_rate_2', 'release_year', 'length', 'length_2', 'replacement_cost', 'deleted_scenes', 'behind_the_scenes', 'NC-17', 'PG', 'PG-13', 'R']
X = df.loc[:, features_to_keep].copy()
y = df['rental_length_days']
print(X.columns)
print(y.value_counts())

Index(['amount', 'amount_2', 'rental_rate', 'rental_rate_2', 'release_year',
       'length', 'length_2', 'replacement_cost', 'deleted_scenes',
       'behind_the_scenes', 'NC-17', 'PG', 'PG-13', 'R'],
      dtype='object')
rental_length_days
7    1832
1    1829
8    1771
5    1767
6    1758
4    1757
2    1713
3    1694
9     894
0     846
Name: count, dtype: int64


In [31]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)
print('X_train', X_train.shape, 'y_train', y_train.shape, 'X_test', X_test.shape, 'y_test', y_test.shape)

X_train (12688, 14) y_train (12688,) X_test (3173, 14) y_test (3173,)


In [32]:
# Predict the labels using Logistic Regression
lr = LogisticRegression(C=100)
lr.fit(X_train, y_train)
lr_y_pred = lr.predict(X_test)
print('LR mean_squared_error', mean_squared_error(y_test, lr_y_pred))

LR mean_squared_error 8.914276709738418


/usr/local/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [33]:
# Predict the labels using Decision Tree Classifier
dtc = DecisionTreeClassifier(max_depth=8, random_state=SEED)
dtc.fit(X_train, y_train)
dtc_y_pred = dtc.predict(X_test)
print('DTC mean_squared_error', mean_squared_error(y_test, dtc_y_pred))

DTC mean_squared_error 3.6876772770248976


In [35]:
# Predict the labels using AdaBoost Classification
abc = AdaBoostClassifier(n_estimators=NESTIM)
abc.fit(X_train, y_train)
abc_y_pred = abc.predict(X_test)
print('ABC mean_squared_error', mean_squared_error(y_test, abc_y_pred))
abc_MSE_CV = - cross_val_score(abc,
                               X_train,
                               y_train,
                               cv=10,
                               scoring='neg_mean_squared_error',
                               n_jobs=-1)
print('ABC_MSE_CV mean_squared_error: {:.2f}'.format(abc_MSE_CV.mean()))

/usr/local/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


ABC mean_squared_error 3.375354554049795


/usr/local/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.1

ABC_MSE_CV mean_squared_error: 3.36


In [36]:
# Predict the labels using Decision Tree Regressor
dtr = DecisionTreeRegressor(max_depth=8, min_samples_leaf=0.1, random_state=SEED)
dtr.fit(X_train, y_train)
dtr_y_pred = dtr.predict(X_test)
print('DTR mean_squared_error', mean_squared_error(y_test, dtr_y_pred))
dtr_MSE_CV = - cross_val_score(dtr, X_train, y_train, cv=10, scoring='neg_mean_squared_error', n_jobs=-1)
print('DTR_MSE_CV mean_squared_error: {:.2f}'.format(dtr_MSE_CV.mean()))

DTR mean_squared_error 3.274658361722676
DTR_MSE_CV mean_squared_error: 3.21


In [37]:
# Predict the labels using Random Forest with GridSearchCV
rf = RandomForestRegressor(random_state=SEED)
params_rf = {
    'n_estimators': np.arange(198,200,1),
    'max_depth': np.arange(12,14,1),
    'max_features': ['log2', 'sqrt']
}
grid_rf = GridSearchCV(estimator=rf,
                      param_grid=params_rf,
                      cv=3,
                      scoring='neg_mean_squared_error',
                      verbose=1,
                      n_jobs=-1)
grid_rf.fit(X_train, y_train)
best_hyperparams_rf = grid_rf.best_params_
print('Best hyperparameters:\n', best_hyperparams_rf)
best_model_rf = grid_rf.best_estimator_
y_pred_rf = best_model_rf.predict(X_test)
mse_test_rf = mean_squared_error(y_test, y_pred_rf)
print('Test set MSE of grid rf: {:.2f}'.format(mse_test_rf))

best_model = grid_rf
best_mse = mse_test_rf

Fitting 3 folds for each of 8 candidates, totalling 24 fits
Best hyperparameters:
 {'max_depth': 13, 'max_features': 'log2', 'n_estimators': 199}
Test set MSE of grid rf: 2.03
